In [11]:
import os

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import imageio.v3 as iio
import os
from datetime import timedelta
import sys
from pathlib import Path 

In [2]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")




In [13]:
def get_saving_frames_durations(metadata, saving_fps, df, delta = 5):
    """Функция, которая возвращает список длительностей, в которые следует сохранять кадры.
    df - инфо по нарушениям 
    delta - сколько секунд отступаем до нарушения
    """

    # получаем продолжительность клипа, разделив количество кадров на количество кадров в секунду
    clip_duration = metadata['duration']
        
    flags = [] # метки флага нарушений
    number_of_violent = [] # номер нарушения в кадре
    descr_list = [] # список описаний кадров
    s = []  # список секунд для стоп-кадров
    end_old = 0
    
    for idx, row in df.iterrows():

        m_start = row.start.minute
        s_start = row.start.second

        m_end = row.end.minute
        s_end = row.end.second

         # преобразуем время нарушений в секунды    
        start = m_start*60 + s_start
        end = m_end*60 + s_end
        
        # определяем время начала нарушения           
        if start-delta < end_old:
            start_new = start
        else: 
            start_new = start-delta
        if  end - start > delta:
            end_new = start+delta
        else:
            end_new = end
            
        descr = row.description
        
        # определяем список кадров    
        for i in np.arange(start_new, end, 1 / saving_fps):
            s.append(i)
            number_of_violent.append(idx)
            if i < start:
                #  до начала нарушения флаг 0           
                flags.append(0)
                descr_list.append(None)
            else:
                flags.append(1)
                descr_list.append(descr)
        end_old = end
    return (s, number_of_violent, flags, descr_list)

In [14]:
def main(video_file, video_name, video_num, df, delta, save_dir, make_dir=False):
    """
    Функция выделяет в видео отрезки с нарушенями и за __ секунд до нарушения,
    создает кадры и сохраняет их в заданную папку.
    
    Args:
        video_file - путь до видео
        video_name - название видео
        video_num - номер видео
        df - данные по нарушениям для конкретного видео
        delta - сколько секунд отступаем до нарушения
        make_dir - сохраняем в одну папку или в разные
        save_dir - куда сохраняем
    """
    file_names = pd.DataFrame()
    
    # создаем папку для сохранения результатов
    if not os.path.isdir(save_dir):
        os.mkdir(save_dir)
        
    filename = save_dir
    
    if make_dir:
        filename, _ = os.path.splitext(video_name)
        filename = os.path.join(save_dir, filename+"_iio")    
        # создаем индивидуальную папку для сохранения результатов
        if not os.path.isdir(filename):
            os.mkdir(filename)
            
    # получить FPS видео
    metadata = iio.immeta(os.path.join(os.getcwd(), 'data', 'train', 'videos', video_name),exclude_applied=False)
    fps = metadata['fps']
    # если SAVING_FRAMES_PER_SECOND выше видео FPS, то установите его на FPS (как максимум)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # получить список длительностей для сохранения
    saving_frames_durations, number_of_violent, flags, descr_list = get_saving_frames_durations(metadata, saving_frames_per_second, df, delta)
    # запускаем цикл
    count = 0
    save_frame_count = 0
    for frame in iio.imiter(os.path.join(os.getcwd(), 'data', 'train', 'videos', video_name), plugin="pyav"):
        # получаем продолжительность, разделив количество кадров на FPS
        frame_duration = count / fps
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # список пуст, все кадры длительности сохранены
            break
        if frame_duration >= closest_duration:

            # если ближайшая длительность меньше или равна длительности кадра,
            # затем сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))           
            
            count_violation = number_of_violent[save_frame_count]
            flag = flags[save_frame_count]
            descr = descr_list[save_frame_count]
            frame_name = f"video_{video_num}_frame_{frame_duration_formatted}_flag_{flag}_violation_{count_violation}.jpg"
#             print(frame_name)

            iio.imwrite(os.path.join(filename, frame_name), frame) 
            file_df = pd.DataFrame({'video_name': video_name,
                                 'video_num': video_num,
                                 'image_name': frame_name,
                                 'frame_duration': frame_duration_formatted,
                                 'target': flag,
                                 'description': descr}, index=[0])
            file_names = pd.concat([file_names, file_df], ignore_index=True)
    
#             file_names.append(frame_name)
            save_frame_count += 1
            # удалить точку продолжительности из списка, так как эта точка длительности уже сохранена
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # увеличить количество кадров
        count += 1
    return file_names

In [15]:
info_df = pd.read_excel(os.path.join(os.getcwd(), 'data', 'train', 'разметка.xlsx'))

In [16]:
info_df

,videofile,start,end,description
0,0000000_00000020240221082923_0001_IMP (1).mp4,00:01:43,00:01:49,Близко стоит к путям при движении поезда
1,0000000_00000020240221082923_0001_IMP (1).mp4,00:06:10,00:06:22,Встал на подножку до полной остановки поезда
2,0000000_00000020240221082923_0001_IMP (1).mp4,00:10:49,00:10:50,Стоит возле поезда при стыковке
3,0000000_00000020240221082923_0001_IMP (1).mp4,00:10:55,00:11:00,Залез под поезд
4,0000000_00000020240221084423_0002_IMP.mp4,00:07:05,00:07:33,Близко стоит к путям при движении поезда
5,0000000_00000020240221084423_0002_IMP.mp4,00:08:10,00:08:15,"достает башмак, залезая под поезд"
6,0000000_00000020240221085923_0003_IMP.mp4,00:09:10,00:12:00,Встал на подножку до полной остановки поезда
7,0000000_00000020240221085923_0003_IMP.mp4,00:13:33,00:13:35,Стоит возле поезда при его движении
8,0000000_00000020240221085923_0003_IMP.mp4,00:13:38,00:13:41,Залез под поезд
9,0000000_00000020240508114229_0002.mp4,00:02:28,00:04:55,стоит слишком близко к поезду движущемуся


In [17]:
SAVING_FRAMES_PER_SECOND = 12 # кол-во кадров  в секунду
DELTA = 10 # отступаем до начала нарушения 5 сек

In [18]:
# info_df = info_df.head()

In [19]:
save_result_path = os.path.join(os.getcwd(), 'data', 'train', 'images')


image_result_info = pd.DataFrame()

for vi, f in tqdm(enumerate(info_df.videofile.unique()), total=info_df.shape[0]):
    temp = info_df[info_df.videofile==f]#.reset_index(drop=True)
    video_file = os.path.join(os.getcwd(), 'data', 'train', 'videos', f'{f}')  
    file_names_df = main(video_file, f, vi, temp, DELTA, save_result_path)   
    image_result_info = pd.concat([image_result_info, file_names_df], ignore_index=True)

 47%|████▋     | 16/34 [23:32<26:29, 88.29s/it]  


In [20]:
image_result_info.to_csv('image_result_info.csv')

In [21]:
image_result_info.target.value_counts()

1    8124
0    3012
Name: target, dtype: int64